# imports and base_dir

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
import os
from keras import optimizers
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, Cropping2D
from keras.layers import MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.layers import GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

from keras.optimizers import Adam

plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

data_dir = '/home/ubuntu/data/iceberg'

import keras
print(keras.__version__)

Using TensorFlow backend.


2.1.1


# create generators from the pngs

In [42]:
train_datagen = ImageDataGenerator(
        #rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=90,
        horizontal_flip=True, 
        preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(#rescale=1./255, 
                                  preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        '/home/ubuntu/data/iceberg/pngs/train',
        target_size=(224, 224),
        batch_size=30,
        shuffle=False,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        '/home/ubuntu/data/iceberg/pngs/valid',
        target_size=(224, 224),
        batch_size=20,
        shuffle=False,
        class_mode='categorical')

Found 1200 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


# precompute the convolutional layers and save 

In [4]:
vgg_base = VGG16(weights='imagenet', include_top=False)

In [12]:
def create_precomputed_data(model, generator):
    filenames = generator.filenames
    conv_features = model.predict_generator(generator, (generator.n/generator.batch_size))
    labels_onehot = to_categorical(generator.classes)
    labels = generator.classes
    return (filenames, conv_features, labels_onehot, labels)

In [43]:
trn_filenames, trn_conv_features, trn_labels, trn_labels_1 = create_precomputed_data(vgg_base, train_generator)
val_filenames, val_conv_features, val_labels, val_labels_1 = create_precomputed_data(vgg_base, validation_generator)

In [44]:
print(trn_conv_features.shape)
print(val_conv_features.shape)

(1200, 7, 7, 512)
(400, 7, 7, 512)


In [6]:
assert len(trn_filenames) == 1200, "trn_filenames not as expected"
assert trn_conv_features.shape == (1200, 7, 7, 512), "trn_conv_features not as expected"
assert trn_labels.shape == (1200, 2), "trn_labels not as expected"

assert len(val_filenames) == 400, "val_filenames not as expected"
assert val_conv_features.shape == (400, 7, 7, 512), "val_conv_features not as expected"
assert val_labels.shape == (400, 2), "val_labels not as expected"

In [3]:
RESULTS_DIR = '/home/ubuntu/data/iceberg/results'

In [47]:
import bcolz
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def save_precomputed_data(filenames, conv_feats, labels, features_base_name="VGG16_conv_feats/trn_"):
    save_array(RESULTS_DIR+"/"+features_base_name+'filenames.dat', np.array(filenames))
    save_array(RESULTS_DIR+"/"+features_base_name+'conv_feats.dat', conv_feats)
    save_array(RESULTS_DIR+"/"+features_base_name+'labels.dat', np.array(labels))
    
save_precomputed_data(trn_filenames, trn_conv_features, trn_labels, "VGG16_conv_feats/trn_")
save_precomputed_data(val_filenames, val_conv_features, val_labels, "VGG16_conv_feats/val_")

In [4]:
import bcolz
def load_array(fname):
    return bcolz.open(fname)[:]

def load_precomputed_data(features_base_name="VGG16_conv_feats/trn_"):
    filenames = load_array(RESULTS_DIR+"/"+features_base_name+'filenames.dat').tolist()
    conv_feats = load_array(RESULTS_DIR+"/"+features_base_name+'conv_feats.dat')
    labels = load_array(RESULTS_DIR+"/"+features_base_name+'labels.dat')
    return filenames, conv_feats, labels

trn_filenames, trn_conv_features, trn_labels = load_precomputed_data("VGG16_conv_feats/trn_")
val_filenames, val_conv_features, val_labels = load_precomputed_data("VGG16_conv_feats/val_")

# create simple model on top of the conv layers

In [15]:
# Create classifier model
p = 0.3 

classifier_input_shape = (7, 7, 512)
classifier_input = Input(shape=classifier_input_shape)

x= Flatten()(classifier_input)
x = Dense(512, activation='relu')(x)
x = Dropout(p)(x)
x = Dense(256, activation='sigmoid')(x)
x = Dropout(p)(x)
x = Dense(2, activation='softmax')(x)
                                                     
classifier_model_v1 = Model(classifier_input, x)

classifier_model_v1.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
from keras.callbacks import TensorBoard
tbCallBack = TensorBoard(log_dir='/home/ubuntu/data/iceberg/tb_logs_ice/', histogram_freq=0, write_graph=True, write_images=True)

In [17]:
classifier_model_v1.fit(trn_conv_features, trn_labels,
                                          batch_size=32, 
                                          epochs=50,
                                          validation_data=(val_conv_features, val_labels),
                                          shuffle=True, 
                                          callbacks=[tbCallBack])

Train on 1200 samples, validate on 400 samples
Epoch 1/50
1200/1200 [==============================] - 1s 837us/step - loss: 0.7329 - acc: 0.5892 - val_loss: 0.6477 - val_acc: 0.5950
Epoch 2/50
1200/1200 [==============================] - 1s 735us/step - loss: 0.6659 - acc: 0.6375 - val_loss: 0.6020 - val_acc: 0.6375
Epoch 3/50
1200/1200 [==============================] - 1s 740us/step - loss: 0.6418 - acc: 0.6408 - val_loss: 0.5882 - val_acc: 0.6200
Epoch 4/50
1200/1200 [==============================] - 1s 737us/step - loss: 0.6154 - acc: 0.6525 - val_loss: 0.5812 - val_acc: 0.6450
Epoch 5/50
1200/1200 [==============================] - 1s 737us/step - loss: 0.5688 - acc: 0.6933 - val_loss: 0.5296 - val_acc: 0.7275
Epoch 6/50
1200/1200 [==============================] - 1s 740us/step - loss: 0.5518 - acc: 0.7100 - val_loss: 0.5517 - val_acc: 0.7175
Epoch 7/50
1200/1200 [==============================] - 1s 733us/step - loss: 0.5086 - acc: 0.7467 - val_loss: 0.4749 - val_acc: 0.7650
E

In [76]:
from keras import backend as K

K.set_value(classifier_model_v1.optimizer.lr, 0.001)
K.eval(classifier_model_v1.optimizer.lr)

0.001

# fully conv network

In [114]:

nf = 128
p = 0. # adding any dropout at all means it doesnt train at all

x = Conv2D(nf,(3,3), activation='relu', padding='same')(classifier_input)
x = Dropout(p)(x)
x = Conv2D(2,(3,3), padding='same')(x)

x = GlobalAveragePooling2D()(x)
x = Activation('softmax')(x)

classifier_model_v2 = Model(classifier_input, x)

classifier_model_v2.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [123]:
K.set_value(classifier_model_v1.optimizer.lr, 0.001)
K.eval(classifier_model_v1.optimizer.lr)

0.001

In [118]:
classifier_model_v2.fit(trn_conv_features, trn_labels, 
                                          batch_size=64, 
                                          epochs=10,
                                          validation_data=(val_conv_features, val_labels),
                                          shuffle=True)

Train on 1200 samples, validate on 400 samples
Epoch 1/10
1200/1200 [==============================] - 0s 302us/step - loss: 0.0143 - acc: 1.0000 - val_loss: 0.9275 - val_acc: 0.7725
Epoch 2/10
1200/1200 [==============================] - 0s 288us/step - loss: 0.0111 - acc: 1.0000 - val_loss: 1.1152 - val_acc: 0.7350
Epoch 3/10
1200/1200 [==============================] - 0s 284us/step - loss: 0.0081 - acc: 1.0000 - val_loss: 1.0190 - val_acc: 0.7650
Epoch 4/10
1200/1200 [==============================] - 0s 283us/step - loss: 0.0064 - acc: 1.0000 - val_loss: 1.0358 - val_acc: 0.7775
Epoch 5/10
1200/1200 [==============================] - 0s 287us/step - loss: 0.0055 - acc: 1.0000 - val_loss: 1.0824 - val_acc: 0.7575
Epoch 6/10
1200/1200 [==============================] - 0s 289us/step - loss: 0.0050 - acc: 1.0000 - val_loss: 1.2013 - val_acc: 0.7375
Epoch 7/10
1200/1200 [==============================] - 0s 286us/step - loss: 0.0043 - acc: 1.0000 - val_loss: 1.1072 - val_acc: 0.7600
E

In [86]:
classifier_model_v2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 7, 7, 512)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 7, 7, 128)         589952    
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 7, 7, 2)           2306      
_________________________________________________________________
dropout_4 (Dropout)          (None, 7, 7, 2)           0         
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2)                 0         
_________________________________________________________________
activation_52 (Activation)   (None, 2)                 0         
Total params: 592,258
Trainable params: 592,258
Non-trainable params: 0
_________________________________________________________________


In [139]:

nf = 128
p = 0. # adding any dropout at all means it doesnt train at all

x = Conv2D(nf,(3,3), activation='relu', padding='same')(classifier_input)
x = BatchNormalization(axis=1)(x)
x = MaxPooling2D()(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
# x = MaxPooling2D()(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
# x = MaxPooling2D()(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)

# x = MaxPooling2D((1,2))(x)
x = Dropout(p)(x)
x = Conv2D(2,(3,3), padding='same')(x)

x = GlobalAveragePooling2D()(x)
x = Activation('softmax')(x)

classifier_model_v3 = Model(classifier_input, x)

classifier_model_v3.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [140]:
K.set_value(classifier_model_v3.optimizer.lr, 0.00001)
K.eval(classifier_model_v3.optimizer.lr)

9.9999997e-06

In [143]:
classifier_model_v3.fit(trn_conv_features, trn_labels, 
                                          batch_size=64, 
                                          epochs=10,
                                          validation_data=(val_conv_features, val_labels),
                                          shuffle=True)

Train on 1200 samples, validate on 400 samples
Epoch 1/10
1200/1200 [==============================] - 4s 3ms/step - loss: 0.1194 - acc: 0.9933 - val_loss: 0.5148 - val_acc: 0.7400
Epoch 2/10
1200/1200 [==============================] - 4s 3ms/step - loss: 0.1083 - acc: 0.9942 - val_loss: 0.5213 - val_acc: 0.7350
Epoch 3/10
1200/1200 [==============================] - 4s 3ms/step - loss: 0.0976 - acc: 0.9967 - val_loss: 0.5166 - val_acc: 0.7325
Epoch 4/10
1200/1200 [==============================] - 4s 3ms/step - loss: 0.0876 - acc: 0.9975 - val_loss: 0.5187 - val_acc: 0.7375
Epoch 5/10
1200/1200 [==============================] - 4s 3ms/step - loss: 0.0795 - acc: 0.9983 - val_loss: 0.5202 - val_acc: 0.7375
Epoch 6/10
1200/1200 [==============================] - 4s 3ms/step - loss: 0.0717 - acc: 0.9992 - val_loss: 0.5219 - val_acc: 0.7400
Epoch 7/10
1200/1200 [==============================] - 4s 3ms/step - loss: 0.0657 - acc: 1.0000 - val_loss: 0.5230 - val_acc: 0.7425
Epoch 8/10
1200